In [ ]:
# Installing libraries
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu
!pip install datasets
!pip install sentence-transformers
##!nvidia-smi 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 30.9 MB/s 
     |████████████████████████████████| 81 kB 13.6 MB/s 
     |████████████████████████████████| 5.8 MB 68.1 MB/s 
     |████████████████████████████████| 132 kB 80.4 MB/s 
     |████████████████████████████████| 182 kB 78.2 MB/s 
     |████████████████████████████████| 213 kB 66.7 MB/s 
     |████████████████████████████████| 127 kB 54.7 MB/s 
     |████████████████████████████████| 7.6 MB 59.9 MB/s 
     |████████████████████████████████| 1.3 MB 59.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 74 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [ ]:
# Connecting to g drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import time

start = time.time()

In [ ]:
# Importing library
import pandas as pd
import re
import time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
# Reading April-June csv
a = pd.read_csv(r'/content/gdrive/MyDrive/iBas-Data/iBAS-Apr-June-2022-Ticket Search Results.csv')
print(a.shape)
# Reading Jan-March csv
b = pd.read_csv(r'/content/gdrive/MyDrive/iBas-Data/iBAS-Jan-Mar-2022-Ticket Search Results.csv')
print(b.shape)
# Reading July-August csv
c = pd.read_csv(r'/content/gdrive/MyDrive/iBas-Data/iBAS-July-Aug-2022-Ticket Search Results.csv')
print(c.shape)

(4934, 15)
(4475, 15)
(3146, 15)


In [ ]:
df1 = a.append(b)
df = df1.append(c)
print(df.shape)

(12555, 15)


In [ ]:
print(df.shape)
# Removing duplicate ticket number and description
df.drop_duplicates(subset=['Ticket Number','Description'],inplace=True)
print(df.shape)

(12555, 15)
(12555, 15)


In [ ]:
# Merging data of all 3 data frames
df.to_csv('Merged_Data.csv',index= False)

In [ ]:
df.rename(columns = {'Title':'Ticket_Title'}, inplace = True)
df.rename(columns = {'Ticket Number':'Ticket_Number'}, inplace = True)
df.rename(columns = {'Issue Type':'Issue_Type'}, inplace = True)
df.rename(columns = {'Sub-Issue Type':'Sub_Issue_Type'}, inplace = True)
df.rename(columns = {'Create Date/Time':'Create_Date/Time'}, inplace = True)
df.rename(columns = {'Complete Date/Time':'Complete_Date/Time'}, inplace = True)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize

# Function to identify REPLY emails
def replacing1(text) :
  count = 0
  removed = []
  tokens = word_tokenize(text)
  for i in range(len(tokens)):
      if tokens[i] == 'RE' or tokens[i] == 're' or tokens[i] == 'Re':
        #print('ok')
        removed.append('1')
      else :
        continue
  return " ".join(removed)

df['Ticket_Title_1'] = df['Ticket_Title'].apply(lambda x: replacing1(x))  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#df = df.head(2)

In [ ]:
names = []
# Extracting name from email id
def extract_name_email(text) :
  e = []
  text = str(text)
  tokens = word_tokenize(text)
  lst = re.findall('\S+@\S+', text)   
  regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
  for i in lst :
    if(re.fullmatch(regex, i)):
      x = i.split("@")
      names.extend(x)
      #print('x',x)
      y = i.split(".")
      names.extend(y)
      #print('y',y)
      if len(y) != 0 :
        for l in y :
          z = l.split("@")
          names.extend(z)
          #print('z',z)
    else :
      continue
  n = list(set(names))
  #print('n',n)
  for i in range(len(tokens)) :
    if tokens[i] in n :
      continue
    else :
      e.append(tokens[i])
  #print(e)
  return " ".join(e)

In [ ]:
# Converting text to lower case
def convert_to_lower(text):
    text = str(text)
    return text.lower()

def remove_voice_questions(text) :
  text = str(text)
  tokens = word_tokenize(text)
  t = []
  c = ['nt','know','me']
  for i in range(len(tokens)) :
    t.append(tokens[i])

  for j in range(len(tokens)) :
    if t[0] == 'your' :
      #if (t[j] == 'impacted') and ((t[j+1] == 'others') or (t[j+1] == 'company') or (t[j+1] == 'nt') or (t[j]=='only' and t[j+1]=='me')) :
      #break 
      index = t.index('impacted')
      t = t[index+2:]
      break
    else :
      continue
  if t is None :
    t = []
    t.append('Empty')

# Customize email -- removing subject, regards
def tok(text):  
    text = str(text)
    tokens = word_tokenize(text)
    #print(tokens)
    c = []
    removed = []
    for i in range(len(tokens)):
        #print(tokens[i])
        if (i+2 < len(tokens)) or (i < len(tokens)) or (i+1 < len(tokens)) :
          #print("(i+2 < len(tokens)) or (i < len(tokens)) or (i+1 < len(tokens))",i+2,i,i+1,tokens[i])
          try :
            if (tokens[i] == 'from' and tokens[i+1] == ':') or (tokens[i]==':' and tokens[i-1] == 'from') or (tokens[i] == 'from' and tokens[i+1] == 'name') :
              continue
            elif (tokens[i] == 'email' and tokens[i+1] == 'body') or (tokens[i-1] == 'email' and tokens[i] == 'body'):
              continue
            elif ((tokens[i] == 'hi' or tokens[i] == 'hello') and (tokens[i+1] == 'team')) or (tokens[i] == 'ty'):
              continue
            elif (tokens[i] == 'good'  and tokens[i+1] == 'morning') or (tokens[i-1] == 'good'  and tokens[i] == 'morning') :
              continue
            elif (tokens[i] == 'hello'  and tokens[i+1] == ',') or (tokens[i] == 'hey') or (tokens[i] == 'name'  and tokens[i+1] == ':') :
              continue
            elif ((tokens[i-1] == 'hi' or tokens[i-1] == 'hello') and (tokens[i] == 'team')) :
              continue
            elif (tokens[i-1] == 'get' and tokens[i] == 'outlook' and tokens[i+1] == 'for' ) :
              continue
            elif (tokens[i] == 'to' and tokens[i+1] == ':') or (tokens[i] == 'cc'):
              continue
            elif (tokens[i] =='cid') :
              break
            elif (tokens[i] == 'regards' or tokens[i] == 'regard' or tokens[i] == 'thanks' or tokens[i] == 'thank') and (tokens[i+1] != 'for') or (tokens[i] == 'get' and tokens[i+1] == 'outlook' and tokens[i+2] == 'for') or (tokens[i] == 'regards' and tokens[i+1] == ','):
              break
            #elif (tokens[i] == 'get' and tokens[i+1] == 'outlook' and tokens[i+2] == 'for') or (tokens[i] == 'regards' and tokens[i+1] == ','):
             # break
            else :
              c.append(tokens[i])
          except :
            continue
        elif (i+2 > len(tokens)) or (i == len(tokens)) or (i+1 > len(tokens)) :
          c.append(tokens[i])
    
    if ('<' in c) and ('>' in c) :
      l = c.index('<')
      j = c.index('>')
      t = c[:l]
      u = c[j+1:]
      c = t + u
      ##print('if')
        
    return " ".join(c)

def remove_urls(text):
    # Use a regular expression to search for URLs
    url_pattern = r'https?://\S+'
    return re.sub(url_pattern, '', text)

def re_num_tt(text) :
  number_pattern = r'[~^0-9]'
  return re.sub(number_pattern, '', text)


# Merging code of Software names
def remove_numbers(text):
    text = str(text)
    number_pattern = r'[~^0-9]'
    tokens = word_tokenize(text)
    #print(tokens)
    #print("len(tokens)",len(tokens))
    t = []
    for i in range(len(tokens)):
      #print(i)
      try :
        if (i == len(tokens)-1) :
          #print("(i == len(tokens)-1)",i)
          t.append(tokens[i])
        elif (tokens[i] == 'office' or  tokens[i] =='office365') and (tokens[i+1] == '365') :
            #print('office365')
            t.append('office365')
        elif (tokens[i] == 'bau') and (tokens[i+1] =='360') :
            t.append('bau360')
        elif (tokens[i] == 'forti' and tokens[i+1] == 'client') :
            #print('forticlient')
            t.append('forticlient')
        elif (tokens[i]=='non') and (tokens[i+1] == 'prod') :
            t.append('nonprod')
        elif (tokens[i] == 'as' and tokens[i+1] == '400') or (tokens[i] == 'as400') :
            #print('as400')
            t.append('as400')    
        elif (tokens[i]== 'e3') or (tokens[i]== 'e3') or (tokens[i]== 'ec2') or (tokens[i]== 'develop03') or (tokens[i]== 'ie5') or (tokens[i]== 'ie10') or (tokens[i]== 'as400'):
            t.append(tokens[i])
        elif (tokens[i]== 'case') or (tokens[i+1]== '360') or (tokens[i]== 'case360') :
            #print('case360')
            t.append('case360')
        elif (tokens[i]== 'db' or tokens[i]== 'data') and (tokens[i+1]== 'base'):
            t.append('database')
        elif (tokens[i]=='non') and (tokens[i]=='prod') :
            t.append('nonprod')
        elif (tokens[i]== 'ms') :
            t.append('microsoft')
        elif (tokens[i]== 'screen') or (tokens[i]== 'shot') :
            t.append('screenshot')
        elif (tokens[i] == 'do') and (tokens[i] == 'nt') :
            t.append('do')
            t.apppend('not')
        elif (tokens[i] == 'pfa') :
            #print('pfa')
            t.append('please')
            t.append('find')
            t.append('attachment')
        elif (tokens[i] == 'eod') :
            #print('eod')
            t.append('end')
            t.append('of')
            t.append('day')
        elif (tokens[i] == 'pw') :
            #print('eod')
            t.append('password')
        elif (tokens[i]== 'five9') or (tokens[i]=='five') or (tokens[i+1]=='9') :
            t.append('five9')
        elif (tokens[i]=='vpn') :
            t.append('vpn')
        elif (tokens[i]== 'app') :
            t.append('application')
        elif (tokens[i]=='vm') :
            t.append('virtual')
            t.append('machine')
        elif (tokens[i]=='appo6') :
            t.append('appo6')
        elif (tokens[i]== 'PW' or tokens[i]=='pw' or tokens[i]=='password') :
            t.append('password')
        elif (tokens[i] == 'tciket') :
            t.append('ticket')
        elif (tokens[i] == 'ca') and (tokens[i] == 'nt') :
            t.append('cannot')
        ## else :
            #without_number = re.sub(pattern=number_pattern, repl="", string=tokens[i])
            #t.append(str(without_number))
        elif tokens[i].isnumeric() and (t[-1] == 'appo6' or t[-1] == 'office365' or t[-1] == 'five9' or t[-1] == 'case360' or t[-1] == 'as400' or t[-1] == 'ec2' or t[-1] == 'bau360' or t[-1] == 'e3' or t[-1] == 'ie5' or t[-1] == 'ie10') :
          continue
        elif (tokens[i] == 'from' and tokens[i+1] == 'name') or (tokens[i-1] == 'from' and tokens[i] == 'name'):
          continue
        else :
          t.append(tokens[i])
          #print("t.append(tokens[i])",tokens[i])
      except :
        continue
    #print(t)
    return " ".join(t)

# Removing function
import string
def remove_punctuation(text):
    # Make a translation table to remove all punctuation symbols
    translator = text.maketrans('', '', string.punctuation)
    # Use the translation table to remove all punctuation symbols from the text
    return text.translate(translator)

In [ ]:
# Replacing office 365 to office365
df['Description_lower'] = df['Description'].apply(lambda x: convert_to_lower(x))   
df['Description_email'] = df['Description_lower'].apply(lambda x: extract_name_email(x))
df['Description_url'] = df['Description_email'].apply(lambda x: remove_urls(x)) 
df['Description_tok'] = df['Description_url'].apply(lambda x: tok(x))
df['Description_punc'] = df['Description_tok'].apply(lambda x: remove_punctuation(x))
df['Description_digits'] = df['Description_punc'].apply(lambda x: remove_numbers(x)) 
##df['Description_1'] = df['Description_digits'].apply(lambda x: remove_voice_questions(x))   

In [ ]:
df['Ticket_Title'] = df['Ticket_Title'].apply(lambda x: convert_to_lower(x))   
df['Ticket_Title'] = df['Ticket_Title'].apply(lambda x: tok(x))
df['Ticket_Title'] = df['Ticket_Title'].apply(lambda x: remove_numbers(x)) 
df['Ticket_Title'] = df['Ticket_Title'].apply(lambda x: re_num_tt(x)) 

In [ ]:
df['Description_email'][1]

1    from : @ from name : to : @ cc : @ email body ...
1    backup job exceeded running time of 24 hrs - p...
1    user is requesting set up for forticlient vpn ...
Name: Description_email, dtype: object

In [ ]:
#df['Description_tok'][1]

In [ ]:
#df['Description_digits'][1]

In [ ]:
print(df.shape)
df.drop_duplicates(subset='Ticket_Number', keep="first")
print(df.shape)

(12555, 22)
(12555, 22)


In [ ]:
print(df.shape)
df =df[df['Ticket_Title_1'] == '']
print(df.shape)

(12555, 22)
(11122, 22)


In [ ]:
# Merging ticket title and description
df['Description_re_concate'] = df['Description_digits'] + ' ' + df['Ticket_Title']

<ipython-input-21-369a5e7b79e2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description_re_concate'] = df['Description_digits'] + ' ' + df['Ticket_Title']


In [ ]:
df.to_csv('Text_Processing.csv',index= False)

In [ ]:
# Pre trained Sentence Transformers
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
import torch

device = torch.device("cuda")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0): MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_features

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
  encoded_input = tokenizer(
  text_list, padding=True, truncation=True, return_tensors="pt")
  encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
  model_output = model(**encoded_input)
  return cls_pooling(model_output)

In [ ]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(df)
comments_dataset

Dataset({
    features: ['Classification', 'Ticket_Number', 'Ticket_Title', 'Description', 'Account', 'Queue', 'Resources', 'Status', 'Priority', 'Created', 'Due', 'Impact', 'Issue_Type', 'Sub_Issue_Type', 'Resolution', 'Ticket_Title_1', 'Description_lower', 'Description_email', 'Description_url', 'Description_tok', 'Description_punc', 'Description_digits', 'Description_re_concate', '__index_level_0__'],
    num_rows: 11122
})

In [ ]:
embedding = get_embeddings(comments_dataset['Description_re_concate'][0])
embedding.shape

torch.Size([1, 768])

In [ ]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["Description_re_concate"]).detach().cpu().numpy()[0]}
)

  0%|          | 0/11122 [00:00<?, ?ex/s]

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/12 [00:00<?, ?it/s]

Dataset({
    features: ['Classification', 'Ticket_Number', 'Ticket_Title', 'Description', 'Account', 'Queue', 'Resources', 'Status', 'Priority', 'Created', 'Due', 'Impact', 'Issue_Type', 'Sub_Issue_Type', 'Resolution', 'Ticket_Title_1', 'Description_lower', 'Description_email', 'Description_url', 'Description_tok', 'Description_punc', 'Description_digits', 'Description_re_concate', '__index_level_0__', 'embeddings'],
    num_rows: 11122
})

In [ ]:
embeddings_dataset

Dataset({
    features: ['Classification', 'Ticket_Number', 'Ticket_Title', 'Description', 'Account', 'Queue', 'Resources', 'Status', 'Priority', 'Created', 'Due', 'Impact', 'Issue_Type', 'Sub_Issue_Type', 'Resolution', 'Ticket_Title_1', 'Description_lower', 'Description_email', 'Description_url', 'Description_tok', 'Description_punc', 'Description_digits', 'Description_re_concate', '__index_level_0__', 'embeddings'],
    num_rows: 11122
})

In [ ]:
#question = "documents not opening in case360"
question = "teams login error"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [ ]:
import pickle
# Save the text embeddings to a file using pickle
with open("text_embeddings.pkl", "wb") as f:
    pickle.dump(embeddings_dataset, f)

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=20
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores

In [ ]:
samples_df.columns

Index(['Classification', 'Ticket_Number', 'Ticket_Title', 'Description',
       'Account', 'Queue', 'Resources', 'Status', 'Priority', 'Created', 'Due',
       'Impact', 'Issue_Type', 'Sub_Issue_Type', 'Resolution',
       'Ticket_Title_1', 'Description_lower', 'Description_email',
       'Description_url', 'Description_tok', 'Description_punc',
       'Description_digits', 'Description_re_concate', '__index_level_0__',
       'embeddings', 'scores'],
      dtype='object')

In [ ]:
samples_df.head(5)

,Classification,Ticket_Number,Ticket_Title,Description,Account,Queue,Resources,Status,Priority,Created,...,Description_lower,Description_email,Description_url,Description_tok,Description_punc,Description_digits,Description_re_concate,__index_level_0__,embeddings,scores
0,CDS-Northwest,T20220822.0393,unable to log in to sharepoint and teams,Unable to log in to Sharepoint and Teams\n\nCB...,"iBAS, Inc (CHCS)",Global Shared Service Desk,"GUPTA, MUKESH (primary)",Complete,P3 - Moderate,8/22/2022 19:26,...,unable to log in to sharepoint and teams\n\ncb...,unable to log to sharepoint and teams cb : 208...,unable to log to sharepoint and teams cb : 208...,unable to log to sharepoint and teams cb : 208...,unable to log to sharepoint and teams cb 2084...,unable to log to sharepoint and teams cb 20840...,unable to log to sharepoint and teams cb 20840...,2669,"[-0.20379874110221863, -0.14271019399166107, -...",14.659274
1,CDS-Northwest,T20220831.0605,unable to sign in to teams,She said password has expired.,"iBAS, Inc (CHCS)",Global Shared Service Desk,"YBARSABAL, RAY (primary)",Complete,P3 - Moderate,8/31/2022 20:40,...,she said password has expired.,she said password has expired .,she said password has expired .,she said password has expired .,she said password has expired,she said password has expired,she said password has expired unable to sign i...,3138,"[-0.06729351729154587, -0.20569178462028503, -...",15.462677
2,CDS-Northwest,T20220302.0188,teams error,FROM: Manish-Kumar.Verma@chcs-services.com\nFR...,"iBAS, Inc (CHCS)",Merged Tickets,"LEDESMA, TONY (primary)",Complete,Scheduled,3/2/2022 18:26,...,from: manish-kumar.verma@chcs-services.com\nfr...,from : @ from name : manish to : @ cc : @ ; @ ...,from : @ from name : manish to : @ cc : @ ; @ ...,"@ : manish : @ : @ ; @ ; @ : , am getting belo...",manish am getting below error whil...,manish am getting below error while login into...,manish am getting below error while login into...,1974,"[-0.21326568722724915, -0.2221154123544693, -0...",15.561868
3,CDS-Northwest,T20220125.0391,not able to login on microsoft teams - #,FROM: servicedeskbpo.global@capgemini.com\nFRO...,"iBAS, Inc (CHCS)",Global Shared Service Desk,"SONI, JITENDRA (primary)",Complete,P2 - Impaired,1/25/2022 21:26,...,from: servicedeskbpo.global@capgemini.com\nfro...,"from : @ from name : bsv it , desk to : @ cc :...","from : @ from name : bsv it , desk to : @ cc :...","@ : bsv it , desk : @ : @ ; @ : , below users ...",bsv it desk below users are not abl...,bsv it desk below users are not able to login ...,bsv it desk below users are not able to login ...,166,"[0.27587559819221497, -0.7293131947517395, -0....",16.956728
4,CDS-Northwest,T20220303.0229,can not access microsoft teams,FROM: kathleen.brown@chcs-services.com\nFROM N...,"iBAS, Inc (CHCS)",CDS-Northwest,"CURRIE, SEAN (primary)",Complete,P4 - Functional,3/3/2022 18:36,...,from: kathleen.brown@chcs-services.com\nfrom n...,from : @ from name : to : @ cc : email body : ...,from : @ from name : to : @ cc : email body : ...,"@ : : @ : : , am receiving the below error mes...",am receiving the below error message wh...,am receiving the below error message when try ...,am receiving the below error message when try ...,2078,"[-0.4583558440208435, -0.2991095185279846, -0....",17.276005


In [ ]:
from numpy.linalg import norm
import numpy as np
cos = []
for i in samples_df['embeddings'] :
  cosine = np.dot(question_embedding,i)/(norm(question_embedding)*norm(i))
  cos.append(cosine)

samples_df['Cosine_Simlarity'] = cos

In [ ]:
samples_df

,Classification,Ticket_Number,Ticket_Title,Description,Account,Queue,Resources,Status,Priority,Created,...,Description_url,Description_tok,Description_punc,Description_digits,Description_re_concate,__index_level_0__,embeddings,scores,Cosine_Score,Cosine_Simlarity
0,CDS-Northwest,T20220822.0393,unable to log in to sharepoint and teams,Unable to log in to Sharepoint and Teams\n\nCB...,"iBAS, Inc (CHCS)",Global Shared Service Desk,"GUPTA, MUKESH (primary)",Complete,P3 - Moderate,8/22/2022 19:26,...,unable to log to sharepoint and teams cb : 208...,unable to log to sharepoint and teams cb : 208...,unable to log to sharepoint and teams cb 2084...,unable to log to sharepoint and teams cb 20840...,unable to log to sharepoint and teams cb 20840...,2669,"[-0.20379874110221863, -0.14271019399166107, -...",14.659274,0.791,[0.7909102335759153]
2,CDS-Northwest,T20220302.0188,teams error,FROM: Manish-Kumar.Verma@chcs-services.com\nFR...,"iBAS, Inc (CHCS)",Merged Tickets,"LEDESMA, TONY (primary)",Complete,Scheduled,3/2/2022 18:26,...,from : @ from name : manish to : @ cc : @ ; @ ...,"@ : manish : @ : @ ; @ ; @ : , am getting belo...",manish am getting below error whil...,manish am getting below error while login into...,manish am getting below error while login into...,1974,"[-0.21326568722724915, -0.2221154123544693, -0...",15.561868,0.781,[0.7807703626189605]
1,CDS-Northwest,T20220831.0605,unable to sign in to teams,She said password has expired.,"iBAS, Inc (CHCS)",Global Shared Service Desk,"YBARSABAL, RAY (primary)",Complete,P3 - Moderate,8/31/2022 20:40,...,she said password has expired .,she said password has expired .,she said password has expired,she said password has expired,she said password has expired unable to sign i...,3138,"[-0.06729351729154587, -0.20569178462028503, -...",15.462677,0.781,[0.7809711789016316]
3,CDS-Northwest,T20220125.0391,not able to login on microsoft teams - #,FROM: servicedeskbpo.global@capgemini.com\nFRO...,"iBAS, Inc (CHCS)",Global Shared Service Desk,"SONI, JITENDRA (primary)",Complete,P2 - Impaired,1/25/2022 21:26,...,"from : @ from name : bsv it , desk to : @ cc :...","@ : bsv it , desk : @ : @ ; @ : , below users ...",bsv it desk below users are not abl...,bsv it desk below users are not able to login ...,bsv it desk below users are not able to login ...,166,"[0.27587559819221497, -0.7293131947517395, -0....",16.956728,0.773,[0.7727711549809403]
5,CDS-Northwest,T20220325.1419,not able to login in teams,None,"iBAS, Inc (CHCS)",Global Shared Service Desk,"PARMAR, NIKUNJ (primary) | PATODI, ANKIT",Complete,P3 - Moderate,3/26/2022 4:23,...,nan,nan,nan,nan,nan not able to login in teams,4208,"[0.027764372527599335, -0.4279092252254486, -0...",17.633350,0.764,[0.7644387541990159]
4,CDS-Northwest,T20220303.0229,can not access microsoft teams,FROM: kathleen.brown@chcs-services.com\nFROM N...,"iBAS, Inc (CHCS)",CDS-Northwest,"CURRIE, SEAN (primary)",Complete,P4 - Functional,3/3/2022 18:36,...,from : @ from name : to : @ cc : email body : ...,"@ : : @ : : , am receiving the below error mes...",am receiving the below error message wh...,am receiving the below error message when try ...,am receiving the below error message when try ...,2078,"[-0.4583558440208435, -0.2991095185279846, -0....",17.276005,0.759,[0.7590956139123137]
6,CDS-Northwest,T20220124.0470,not able to login on microsoft teams,FROM: amit.kashyap@chcs-services.com\nFROM NAM...,"iBAS, Inc (CHCS)",Global Shared Service Desk,"YBARSABAL, RAY (primary)",Complete,P3 - Moderate,1/24/2022 21:47,...,from : @ from name : to : @ ; @ ; dmodi @ cc :...,@ : : @ ; @ ; dmodi @ : @ ; @ ; @ ; @ ; hemend...,dmodi hemendradeopa below ...,dmodi hemendradeopa below user is not able to ...,dmodi hemendradeopa below user is not able to ...,153,"[0.12157537043094635, -0.4818180203437805, -0....",17.767136,0.759,[0.7585010099855292]
7,CDS-Northwest,T20220308.1720,teams issue,Teams Issue\nCB:-8505541324,"iBAS, Inc (CHCS)",Global Shared Service Desk,"PATODI, ANKIT (primar

In [ ]:
from scipy.spatial.distance import cosine
cs = []
for i in samples_df['embeddings'] :
  distance = cosine(question_embedding, i)
  cs.append(np.round(1-distance, 3))

samples_df["Cosine_Score"] = cs
samples_df.sort_values("Cosine_Score", ascending=False, inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    print(f"DESC: {row.Description_re_concate}")
    print(f"FAISS SCORE: {row.scores}")
    print(f"COSINE SCORE: {row.Cosine_Score}")
    print(f"Queue: {row.Queue}")
    print(f"CREATED DATE: {row.Created}")
    print(f"TICKET NUMBER: {row.Ticket_Number}")
    print("=" * 50)
    print()

DESC: unable to log to sharepoint and teams cb 2084098490 unable to log in to sharepoint and teams
FAISS SCORE: 14.659274101257324
COSINE SCORE: 0.791
Queue: Global Shared Service Desk
CREATED DATE: 8/22/2022 19:26
TICKET NUMBER: T20220822.0393

DESC: manish am getting below error while login into teams on new laptop its asking for authenticator code kindly fix this issue and resolve teams error
FAISS SCORE: 15.561867713928223
COSINE SCORE: 0.781
Queue: Merged Tickets
CREATED DATE: 3/2/2022 18:26
TICKET NUMBER: T20220302.0188

DESC: she said password has expired unable to sign in to teams
FAISS SCORE: 15.462677001953125
COSINE SCORE: 0.781
Queue: Global Shared Service Desk
CREATED DATE: 8/31/2022 20:40
TICKET NUMBER: T20220831.0605

DESC: bsv it desk below users are not able to login on teams please have a look and fix the issue on priority please change the authentication method to text message for him email hemendradeopa phone no 7042611540 email mailto phone no 8447739983 not able t